# FINAL PROJECT: ÍÑIGO BARCELÓ ÁLVAREZ / JUAN LEAL ALIAGA

## 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data or newtype) for each of them. 

In [36]:
data Date = Date 
  { day   :: Int
  , month :: String
  , year  :: Int
  } deriving (Show, Eq, Ord)

data Person = Person 
  { firstName :: String
  , lastName  :: String
  , birthDate :: Date
  } deriving (Show, Eq)

data Publication
  = Book
      { bookId    :: String       -- Unique signature
      , author    :: Person       -- Single author
      , title     :: String
      , pages     :: Int
      , copies    :: [Copy]       -- List of copies
      }
  | Journal
      { journalId :: String       -- Unique signature
      , authors   :: [Person]     -- Multiple authors
      , title     :: String
      , releaseDate :: Date
      , pages     :: Int
      , copies    :: [Copy]
      }
  | DVD
      { dvdId     :: String       -- Unique signature
      , director  :: Person       -- Director is a single person
      , actors    :: [Person]     -- List of actors
      , title     :: String
      , releaseDate :: Date
      , duration  :: Int          -- Duration in minutes
      , copies    :: [Copy]
      }
  deriving (Show, Eq)

data Copy = Copy
  { isBorrowed  :: Bool
  , borrower    :: Maybe User    -- User who borrowed the copy (if any)
  , loanDate    :: Maybe Date    -- Loan date (if borrowed)
  , returnDate  :: Maybe Date    -- Expected return date (if borrowed)
  } deriving (Show, Eq)

data User
  = Student 
      { studentName :: Person
      }
  | Professor
      { professorName :: Person
      }
  
  deriving (Show, Eq)

type Catalog = [Publication]

## 2. Functions to show the former concepts in a pretty way. 

In [37]:
instance Show Date where
  show (Date d m y) = show d ++ " " ++ m ++ " " ++ show y

instance Show Person where
  show (Person fName lName bDate) = 
    fName ++ " " ++ lName ++ " (" ++ show bDate ++ ")"
    
instance Show Copy where
  show (Copy isB mbUser mbLoanDate mbReturnDate)
    | isB = "Borrowed by " ++ maybe "Unknown" show mbUser ++
            " | Loan Date: " ++ maybe "N/A" show mbLoanDate ++
            " | Return Date: " ++ maybe "N/A" show mbReturnDate
    | otherwise = "Available"
    
instance Show Publication where
  show (Book id author title pages copies) =
    "Book [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Author: " ++ show author ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (Journal id authors title releaseDate pages copies) =
    "Journal [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Authors: " ++ unwords (map show authors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (DVD id director actors title releaseDate duration copies) =
    "DVD [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Director: " ++ show director ++ "\n" ++
    "  Actors: " ++ unwords (map show actors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Duration: " ++ show duration ++ " minutes\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)
    
instance Show User where
    show (Student p) = 
        "Student: " ++ show p ++ "\n"
    
    show (Professor p) = 
        "Professor: " ++ show p ++ "\n"
    
instance Eq User where
    (Student p1) == (Student p2) = p1 == p2
    (Professor p1) == (Professor p2) = p1 == p2
    _ == _ = False
    
prettyCatalog :: Catalog -> String
prettyCatalog [] = "Empty Catalog"
prettyCatalog catalog = unlines $ "Catalog:" : map show catalog



In [38]:
--Prueba
exampleDate :: Date
exampleDate = Date 18 "November" 2024

author1 :: Person
author1 = Person "John" "Doe" (Date 10 "January" 1980)

exampleBook :: Publication
exampleBook = Book 
  "B001" 
  author1 
  "Functional Programming in Haskell" 
  300 
  [Copy False Nothing Nothing Nothing]

exampleStudent :: User
exampleStudent = Student (Person "Alice" "Smith" (Date 5 "May" 2001))

exampleCatalog :: Catalog
exampleCatalog = [exampleBook]

exampleDate

author1

exampleBook

exampleStudent

exampleCatalog


18 November 2024

John Doe (10 January 1980)

Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available

Student: Alice Smith (5 May 2001)

[Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available
]

In [39]:
-- Ejemplo de fecha
exampleLoanDate :: Date
exampleLoanDate = Date 10 "November" 2024

exampleReturnDate :: Date
exampleReturnDate = Date 10 "December" 2024

-- Ejemplo de personas
student1 :: User
student1 = Student (Person "Alice" "Smith" (Date 5 "May" 2001))
professor1 :: User
professor1 = Professor (Person "Dr." "Johnson" (Date 12 "August" 1975)) 
-- Ejemplo de copias de un libro
exampleCopies :: [Copy]
exampleCopies = 
  [ Copy True (Just student1) (Just exampleLoanDate) (Just exampleReturnDate)  -- Prestada a un estudiante
  , Copy True (Just professor1) (Just exampleLoanDate) (Just exampleReturnDate) -- Prestada a un profesor
  , Copy False Nothing Nothing Nothing  -- Disponible
  , Copy False Nothing Nothing Nothing  -- Disponible
  ]

-- Ejemplo de libro con varias copias
exampleBookWithCopies :: Publication
exampleBookWithCopies = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  exampleCopies

-- Catálogo con el libro
exampleCatalogWithCopies :: Catalog
exampleCatalogWithCopies = [exampleBookWithCopies]

exampleBookWithCopies

exampleCatalogWithCopies

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

In [40]:
-- Ejemplo de otro libro con copias
exampleCopies2 :: [Copy]
exampleCopies2 = 
  [ Copy True (Just student1) (Just (Date 1 "November" 2024)) (Just (Date 8 "November" 2024)) -- Prestada a un estudiante
  , Copy False Nothing Nothing Nothing  -- Disponible
  , Copy False Nothing Nothing Nothing  -- Disponible
  ]

exampleBookWithCopies2 :: Publication
exampleBookWithCopies2 = Book
  "B003"
  (Person "George" "Orwell" (Date 25 "June" 1903))
  "1984"
  328
  exampleCopies2

-- Catálogo actualizado con ambos libros
updatedCatalog :: Catalog
updatedCatalog = exampleCatalogWithCopies ++ [exampleBookWithCopies2]

-- Prueba de impresión del catálogo actualizado
prettyCatalog updatedCatalog


"Catalog:\nBook [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 2/4 available\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\n"

In [41]:
prettyCatalog []

"Empty Catalog"

## 3. An addToCatalog Publication Catalog function that adds a publication to the library catalog. If the publication is already in the catalog, its number of copies will be increased. 

In [93]:
addToCatalog :: Publication -> Catalog -> Catalog
addToCatalog pub [] = [pub]
addToCatalog pub (p:ps)
  | samePublication pub p = mergeCopies pub p : ps -- Si ya existe, combina las copias
  | otherwise             = p : addToCatalog pub ps -- Si no, sigue buscando
  where
    -- Verifica si dos publicaciones son la misma basándose en su ID único
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Combina las copias de dos publicaciones iguales
    mergeCopies :: Publication -> Publication -> Publication
    mergeCopies (Book id1 author1 title1 pages1 copies1) (Book _ _ _ _ copies2) =
      Book id1 author1 title1 pages1 (copies1 ++ copies2)
    mergeCopies (Journal id1 authors1 title1 release1 pages1 copies1) (Journal _ _ _ _ _ copies2) =
      Journal id1 authors1 title1 release1 pages1 (copies1 ++ copies2)
    mergeCopies (DVD id1 director1 actors1 title1 release1 duration1 copies1) (DVD _ _ _ _ _ _ copies2) =
      DVD id1 director1 actors1 title1 release1 duration1 (copies1 ++ copies2)
    mergeCopies pub1 _ = pub1 -- Por seguridad, devuelve pub1 en otros casos

-- Nueva copia para un libro existente
newCopiesForPrideAndPrejudice :: [Copy]
newCopiesForPrideAndPrejudice = 
  [ Copy False Nothing Nothing Nothing,  -- Nueva copia disponible
    Copy False Nothing Nothing Nothing   -- Otra nueva copia disponible
  ]

-- Publicación con nuevas copias de "Pride and Prejudice"
newPrideAndPrejudice :: Publication
newPrideAndPrejudice = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  newCopiesForPrideAndPrejudice

-- Nuevo libro completamente nuevo
newBook :: Publication
newBook = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  [Copy False Nothing Nothing Nothing]

-- Actualizar el catálogo
updatedCatalog2 :: Catalog
updatedCatalog2 = addToCatalog newPrideAndPrejudice $ addToCatalog newBook updatedCatalog

-- Prueba de impresión
prettyCatalog updatedCatalog2


"Catalog:\nBook [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 4/6 available\nAvailable\nAvailable\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\nBook [B004]\n  Title: Harry Potter and the Philosopher's Stone\n  Author: J.K. Rowling (31 July 1965)\n  Pages: 223\n  Copies: 1/1 available\nAvailable\n\n"

In [43]:
-- Nuevas copias para "Harry Potter and the Philosopher's Stone"
newCopiesForHarryPotter :: [Copy]
newCopiesForHarryPotter = 
  [ Copy False Nothing Nothing Nothing,  -- Nueva copia disponible
    Copy False Nothing Nothing Nothing   -- Otra nueva copia disponible
  ]

-- Publicación con nuevas copias de "Harry Potter"
newBook2 :: Publication
newBook2 = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  newCopiesForHarryPotter

-- Actualizar el catálogo con newBook2 (se fusionan las copias)
finalCatalog :: Catalog
finalCatalog = addToCatalog newBook2 updatedCatalog2

-- Prueba de impresión
prettyCatalog finalCatalog
"-----Empty Catalog--------"
addToCatalog newBook2 []

"Catalog:\nBook [B002]\n  Title: Pride and Prejudice\n  Author: Jane Austen (16 December 1775)\n  Pages: 432\n  Copies: 4/6 available\nAvailable\nAvailable\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nBorrowed by Professor: Dr. Johnson (12 August 1975)\n | Loan Date: 10 November 2024 | Return Date: 10 December 2024\nAvailable\nAvailable\n\nBook [B003]\n  Title: 1984\n  Author: George Orwell (25 June 1903)\n  Pages: 328\n  Copies: 2/3 available\nBorrowed by Student: Alice Smith (5 May 2001)\n | Loan Date: 1 November 2024 | Return Date: 8 November 2024\nAvailable\nAvailable\n\nBook [B004]\n  Title: Harry Potter and the Philosopher's Stone\n  Author: J.K. Rowling (31 July 1965)\n  Pages: 223\n  Copies: 3/3 available\nAvailable\nAvailable\nAvailable\n\n"

"-----Empty Catalog--------"

[Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 2/2 available
Available
Available
]

## 4. A removeFromCatalog Publication Catalog that removes a copy of the publication from the library catalog. If it is the last copy, the publication will be removed. If the catalog does not contain that publication an error will be raised.

In [44]:
removeFromCatalog :: Publication -> Catalog -> Either String Catalog
removeFromCatalog pub [] = Left "Error: The publication is not in the catalog."
removeFromCatalog pub (p:ps)
  | samePublication pub p =
      let updatedPub = removeCopy pub
      in if noCopiesLeft updatedPub
         then Right ps -- Elimina la publicación completamente
         else Right (updatedPub : ps) -- Actualiza la publicación
  | otherwise = fmap (p :) (removeFromCatalog pub ps) -- Sigue buscando
  where
    -- Verifica si dos publicaciones son iguales por su ID único
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Elimina una copia de la publicación (la primera disponible o prestada)
    removeCopy :: Publication -> Publication
    removeCopy (Book id1 author1 title1 pages1 copies1) =
      Book id1 author1 title1 pages1 (tail copies1)
    removeCopy (Journal id1 authors1 title1 release1 pages1 copies1) =
      Journal id1 authors1 title1 release1 pages1 (tail copies1)
    removeCopy (DVD id1 director1 actors1 title1 release1 duration1 copies1) =
      DVD id1 director1 actors1 title1 release1 duration1 (tail copies1)
    removeCopy pub = pub -- Por seguridad

    -- Verifica si no quedan copias en la publicación
    noCopiesLeft :: Publication -> Bool
    noCopiesLeft (Book _ _ _ _ copies1)    = null copies1
    noCopiesLeft (Journal _ _ _ _ _ copies1) = null copies1
    noCopiesLeft (DVD _ _ _ _ _ _ copies1) = null copies1


In [45]:
-- Catálogo inicial

catalogForRemoval :: Catalog
catalogForRemoval = finalCatalog
finalCatalog
exampleBookWithCopies
-- Eliminar una copia de "Pride and Prejudice"
result1 :: Either String Catalog
result1 = removeFromCatalog exampleBookWithCopies catalogForRemoval
result1
-- Eliminar "1984" completamente
{-result2 :: Either String Catalog
result2 = removeFromCatalog (Book "B003" undefined undefined undefined undefined) <$> result1
result2
-- Intentar eliminar una publicación inexistente
result3 :: Either String Catalog
result3 = removeFromCatalog (Book "B999" undefined undefined undefined undefined) <$> result2
-}
-- Prueba de impresión de resultados
printCatalogResult :: Either String Catalog -> IO ()
printCatalogResult (Left err) = putStrLn $ "Error: " ++ err
printCatalogResult (Right catalog) = putStrLn $ prettyCatalog catalog


   --result1
  --t (removeFromCatalog exampleBookWithCopies catalogForRemoval) 
  --printCatalogResult result1
  
  --printCatalogResult result3


[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 4/6 available
Available
Available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
,Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
Available
Available
,Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
Available
Available
Available
]

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

Right [Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/3 available
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
,Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
Available
Available
,Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
Available
Available
Available
]

In [46]:
exampleBookWithCopies
"---------------------------------"
exampleCatalogWithCopies
"---------------------------------"
removeFromCatalog exampleBookWithCopies exampleCatalogWithCopies
"---------------------------------"
removeFromCatalog exampleBookWithCopies []

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available

"---------------------------------"

[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/4 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

"---------------------------------"

Right [Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/3 available
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
]

"---------------------------------"

Left "Error: The publication is not in the catalog."

## 5. A borrowedPublications User Catalog function that returns the publications the user has borrowed. They must be sort by closest return date, in case of tie in return date, it will first show books, then journals and finally DVDs. 

In [47]:
-- Función principal para obtener las publicaciones prestadas por un usuario
borrowedPublications :: User -> Catalog -> [Publication]
borrowedPublications user catalog = 
  let borrowedCopies = [ publication | publication <- catalog, 
                                     any (isBorrowedByUser user) (copies publication) ]
  in sortPublications borrowedCopies

-- Función auxiliar para verificar si una copia está prestada al usuario
isBorrowedByUser :: User -> Copy -> Bool
isBorrowedByUser user copy = 
  case borrower copy of
    Just u  -> u == user  -- Si el prestatario es el usuario, es una copia tomada por él.
    Nothing -> False      -- Si no está prestada, no la consideramos.

-- Función para ordenar las publicaciones
sortPublications :: [Publication] -> [Publication]
sortPublications [] = []
sortPublications (x:xs) = insertPublication x (sortPublications xs)

-- Función para insertar una publicación en el lugar adecuado
insertPublication :: Publication -> [Publication] -> [Publication]
insertPublication pub [] = [pub]
insertPublication pub (x:xs)
  | comparator pub x == LT = pub : x : xs  -- Si pub debe ir antes de x
  | otherwise = x : insertPublication pub xs  -- Sino, seguimos buscando

-- Comparador para ordenar publicaciones
comparator :: Publication -> Publication -> Ordering
comparator pub1 pub2 = 
  case compareReturnDate pub1 pub2 of
    EQ -> compareType pub1 pub2
    result -> result

-- Función para comparar las fechas de retorno
compareReturnDate :: Publication -> Publication -> Ordering
compareReturnDate pub1 pub2 =
  let returnDate1 = minimum (map returnDate (copies pub1))
      returnDate2 = minimum (map returnDate (copies pub2))
  in compare returnDate1 returnDate2

-- Función para comparar el tipo de publicación (libro primero, luego revista, luego DVD)
compareType :: Publication -> Publication -> Ordering
compareType (Book _ _ _ _ _) (Book _ _ _ _ _) = EQ
compareType (Book _ _ _ _ _) _ = LT
compareType _ (Book _ _ _ _ _) = GT
compareType (Journal _ _ _ _ _ _) (Journal _ _ _ _ _ _) = EQ
compareType (Journal _ _ _ _ _ _) _ = LT
compareType _ (Journal _ _ _ _ _ _) = GT
compareType (DVD _ _ _ _ _ _ _) (DVD _ _ _ _ _ _ _) = EQ
compareType (DVD _ _ _ _ _ _ _) _ = LT
compareType _ (DVD _ _ _ _ _ _ _) = GT

Line 17: Use foldr
Found:
sortPublications [] = []
sortPublications (x : xs)
  = insertPublication x (sortPublications xs)
Why not:
sortPublications xs = foldr insertPublication [] xsLine 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 44: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 45: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 47: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 48: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 50: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 51: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}

In [48]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

testUser = Student 
  (Person "John" "Doe" (Date 1 "January" 2000)) 


borrowedCopy1 = Copy True (Just testUser) (Just (Date 20 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy2 = Copy True (Just testUser) (Just (Date 15 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy3 = Copy True (Just testUser) (Just (Date 18 "November" 2024)) (Just (Date 28 "November" 2024))
borrowedCopy4 = Copy True (Just testUser) (Just (Date 24 "November" 2024)) (Just (Date 28 "November" 2024))
borrowedCopy5 = Copy False Nothing Nothing Nothing
book1, book2, journal1 :: Publication
book1 = Book "B001" (Person "Author1" "Surname1" (Date 1 "January" 1970)) 
                   "Functional Programming in Haskell" 300 [borrowedCopy1]
book2 = Book "B002" (Person "Author2" "Surname2" (Date 1 "February" 1980)) 
                   "Learn You a Haskell" 250 [borrowedCopy2]
journal1 = Journal "J001" [Person "Author3" "Surname3" (Date 1 "March" 1990)] 
                         "Advanced Haskell Research" (Date 1 "March" 2024) 
                         50 [borrowedCopy3, borrowedCopy5]
--dvd1 = DVD "D001" testUser [testUser] "Literature on Screen" (Date 1 "April" 2024) 120 []
dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 [borrowedCopy4]

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]

In [49]:
testUser
print("-----------------------------------")
testCatalog
print("-----------------------------------")
borrowedPublications testUser testCatalog

Line 2: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"Line 4: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"

Student: John Doe (1 January 2000)

"-----------------------------------"

[Book [B001]
  Title: Functional Programming in Haskell
  Author: Author1 Surname1 (1 January 1970)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 20 November 2024 | Return Date: 30 November 2024
,Book [B002]
  Title: Learn You a Haskell
  Author: Author2 Surname2 (1 February 1980)
  Pages: 250
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 15 November 2024 | Return Date: 30 November 2024
,Journal [J001]
  Title: Advanced Haskell Research
  Authors: Author3 Surname3 (1 March 1990)
  Release Date: 1 March 2024
  Pages: 50
  Copies: 1/2 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 18 November 2024 | Return Date: 28 November 2024
Available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/1 available
Borrowed

"-----------------------------------"

[Journal [J001]
  Title: Advanced Haskell Research
  Authors: Author3 Surname3 (1 March 1990)
  Release Date: 1 March 2024
  Pages: 50
  Copies: 1/2 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 18 November 2024 | Return Date: 28 November 2024
Available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 24 November 2024 | Return Date: 28 November 2024
,Book [B002]
  Title: Learn You a Haskell
  Author: Author2 Surname2 (1 February 1980)
  Pages: 250
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 15 November 2024 | Return Date: 30 November 2024
,Book [B001]
  Title: Functional Programming in Haskell
  Author: Author1 Surname1 (1 January 1970)
  Pages: 300
  Copies: 0/1 available
Borrowed

In [50]:
borrowedPublications testUser []


[]

## 6. A publicationsByAuthor Author Catalog that returns a list with all the publications for that author in the Catalog or the empty list if that person has not authored any publication. Create also equivalent booksByAuthor, journalsByAuthor and DVDsByAuthor that restrict the search to books, journals and DVDs respectively. Notice in DVD both directors and actors must be considered. 

In [51]:
-- General function to find all publications by a given author
publicationsByAuthor :: Person -> Catalog -> [Publication]
publicationsByAuthor author catalog =
  filter authoredBy catalog
  where
    -- Checks if a publication is authored by the given person
    authoredBy :: Publication -> Bool
    authoredBy (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredBy (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredBy (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    
booksByAuthor :: Person -> Catalog -> [Publication]
booksByAuthor author catalog =
  filter authoredByBook catalog
  where
    -- Checks if a book is authored by the given person
    authoredByBook :: Publication -> Bool
    authoredByBook (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredByBook _ = False

journalsByAuthor :: Person -> Catalog -> [Publication]
journalsByAuthor author catalog =
  filter authoredByJournal catalog
  where
    -- Checks if a journal is authored by the given person
    authoredByJournal :: Publication -> Bool
    authoredByJournal (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredByJournal _ = False

dvdsByAuthor :: Person -> Catalog -> [Publication]
dvdsByAuthor author catalog =
  filter authoredByDVD catalog
  where
    -- Checks if a DVD is directed or acted by the given person
    authoredByDVD :: Publication -> Bool
    authoredByDVD (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    authoredByDVD _ = False


Line 3: Eta reduce
Found:
publicationsByAuthor author catalog = filter authoredBy catalog
Why not:
publicationsByAuthor author = filter authoredByLine 13: Eta reduce
Found:
booksByAuthor author catalog = filter authoredByBook catalog
Why not:
booksByAuthor author = filter authoredByBookLine 22: Eta reduce
Found:
journalsByAuthor author catalog = filter authoredByJournal catalog
Why not:
journalsByAuthor author = filter authoredByJournalLine 31: Eta reduce
Found:
dvdsByAuthor author catalog = filter authoredByDVD catalog
Why not:
dvdsByAuthor author = filter authoredByDVD

In [52]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]


In [53]:
publicationsByAuthor author1 testCatalog
"------------------------------------"
booksByAuthor author1 testCatalog
"------------------------------------"
journalsByAuthor author1 testCatalog
"------------------------------------"
dvdsByAuthor author2 testCatalog
"------------------------------------"
publicationsByAuthor author1 []
"------------------------------------"
booksByAuthor author1 []
"------------------------------------"
journalsByAuthor author1 []
"------------------------------------"
dvdsByAuthor author2 []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
,Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"------------------------------------"

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

...

[]

"------------------------------------"

[]

"------------------------------------"

[]

## 7. Functions publicationsByTitle String Catalog and publicationsByDate Date Catalog that return all the publications with that title or release date. 

In [54]:
-- Function to find all publications with the given title
publicationsByTitle :: String -> Catalog -> [Publication]
publicationsByTitle title catalog =
  filter hasTitle catalog
  where
    -- Checks if a publication has the given title
    hasTitle :: Publication -> Bool
    hasTitle (Book _ _ pubTitle _ _) = pubTitle == title
    hasTitle (Journal _ _ pubTitle _ _ _) = pubTitle == title
    hasTitle (DVD _ _ _ pubTitle _ _ _) = pubTitle == title

-- Function to find all publications with the given release date
publicationsByDate :: Date -> Catalog -> [Publication]
publicationsByDate date catalog =
  filter hasReleaseDate catalog
  where
    -- Checks if a publication has the given release date
    hasReleaseDate :: Publication -> Bool
    hasReleaseDate (Book _ _ _ _ _) = False  -- Books don't have a specific release date in this case
    hasReleaseDate (Journal _ _ _ pubReleaseDate _ _) = pubReleaseDate == date
    hasReleaseDate (DVD _ _ _ _ pubReleaseDate _ _) = pubReleaseDate == date



Line 3: Eta reduce
Found:
publicationsByTitle title catalog = filter hasTitle catalog
Why not:
publicationsByTitle title = filter hasTitleLine 14: Eta reduce
Found:
publicationsByDate date catalog = filter hasReleaseDate catalog
Why not:
publicationsByDate date = filter hasReleaseDateLine 19: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

In [55]:
let catalog = [book1, journal1, dvd1] -- Replace with your actual catalog of publications

-- Find all publications with the title "Haskell Programming"
let publicationsWithTitle = publicationsByTitle "Pride and Prejudice" catalog
print publicationsWithTitle
"----------------------------------------------------------------------"
-- Find all publications released on a specific date (e.g., "1 Dec 2024")
let date = Date 1 "January" 2024
let publicationsWithDate = publicationsByDate date catalog
print publicationsWithDate
"----------------------------------------------------------------------"
publicationsByDate date []
"----------------------------------------------------------------------"
publicationsByTitle "Pride and Prejudice" []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"----------------------------------------------------------------------"

[]

"----------------------------------------------------------------------"

[]

## 8. A search Author Title Date function that can receive the author, the title and/or the date and returns all the publications matching the criteria. Notice that the function must work for any combination of the three items (for example searching just by author, or by author and date) and that some, or even the three of them, could be empty.

In [56]:
searchAuthorTitleDate :: Maybe Person -> Maybe String -> Maybe Date -> Catalog -> [Publication]
searchAuthorTitleDate author title date catalog = filter matchesCriteria catalog
  where
    -- Checks if a publication matches the given criteria
    matchesCriteria :: Publication -> Bool
    matchesCriteria pub =
      (authorMatches pub) && (titleMatches pub) && (dateMatches pub)

    -- Check if the publication matches the author criterion
    authorMatches :: Publication -> Bool
    authorMatches (Book _ bookAuthor _ _ _) = maybe True (== bookAuthor) author
    authorMatches (Journal _ journalAuthors _ _ _ _) = maybe True (`elem` journalAuthors) author
    authorMatches (DVD _ director actors _ _ _ _) = maybe True (== director) author || maybe False (`elem` actors) author

    -- Check if the publication matches the title criterion
    titleMatches :: Publication -> Bool
    titleMatches (Book _ _ bookTitle _ _) = maybe True (== bookTitle) title
    titleMatches (Journal _ _ journalTitle _ _ _) = maybe True (== journalTitle) title
    titleMatches (DVD _ _ _ dvdTitle _ _ _) = maybe True (== dvdTitle) title

    -- Check if the publication matches the date criterion
    dateMatches :: Publication -> Bool
    dateMatches (Book _ _ _ _ _) = True  -- Books are not filtered by date in this case
    dateMatches (Journal _ _ _ journalDate _ _) = maybe True (== journalDate) date
    dateMatches (DVD _ _ _ _ dvdDate _ _) = maybe True (== dvdDate) date


Line 2: Eta reduce
Found:
searchAuthorTitleDate author title date catalog
  = filter matchesCriteria catalog
Why not:
searchAuthorTitleDate author title date = filter matchesCriteriaLine 7: Redundant bracket
Found:
(authorMatches pub) && (titleMatches pub) && (dateMatches pub)
Why not:
authorMatches pub && (titleMatches pub) && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
titleMatches pub && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
(titleMatches pub) && dateMatches pubLine 23: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

In [57]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]
let catalog = [book1, journal1, dvd1] -- Replace with actual catalog

-- Search for publications by "John Doe" (author), with title "Haskell Programming"
let results = searchAuthorTitleDate (Just (Person "Jane" "Austen" (Date 16 "December" 1775))) (Just "Pride and Prejudice") Nothing catalog
print results
"----------------------------------------------------"
-- Search for publications with the title "Haskell Programming", regardless of author or date
let resultsByTitle = searchAuthorTitleDate Nothing (Just "Victorian Literature") Nothing catalog
print resultsByTitle
"----------------------------------------------------"
-- Search for publications by author "John Doe" released on a specific date
let resultsByAuthorAndDate = searchAuthorTitleDate (Just (Person "Charles" "Dickens" (Date 7 "February" 1812))) Nothing (Just (Date 1 "April" 2024)) catalog
print resultsByAuthorAndDate
"----------------------------------------------------"
searchAuthorTitleDate Nothing Nothing Nothing catalog
"----------------------------------------------------"
searchAuthorTitleDate Nothing Nothing Nothing []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"----------------------------------------------------"

[DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

"----------------------------------------------------"

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
,Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

"----------------------------------------------------"

[]

## 9. A nextWorkingDate days Date [Date] function that receives a positive number of days, a Date and a list of holidays and returns the next working day (not in weekend or holiday) after those days. It will be used to calculate loan and return dates for publications, as the library is closed during weekends. Consider leap years and the fact that 1st January 2025 will be Wednesday. 

In [58]:
-- Mapeo de meses y días
months :: [(String, Int)]
months =
  [ ("January", 1), ("February", 2), ("March", 3), ("April", 4),
    ("May", 5), ("June", 6), ("July", 7), ("August", 8),
    ("September", 9), ("October", 10), ("November", 11), ("December", 12) ]

-- Días por mes considerando años bisiestos
daysInMonth :: String -> Int -> Int
daysInMonth "February" year = if isLeapYear year then 29 else 28
daysInMonth month _
  | month `elem` ["April", "June", "September", "November"] = 30
  | otherwise = 31

isLeapYear :: Int -> Bool
isLeapYear year
  | year `mod` 400 == 0 = True
  | year `mod` 100 == 0 = False
  | year `mod` 4 == 0 = True
  | otherwise = False

-- Función para avanzar días
addDays :: Int -> Date -> Date
addDays 0 date = date
addDays n (Date d m y)
  | n + d <= daysInMonth m y = Date (d + n) m y -- El día resultante cae dentro del mes actual
  | otherwise =
      let daysLeft = n - (daysInMonth m y - d) -- Días que sobran después de llenar el mes actual
          (nextMonth, nextYear) = nextMonthYear m y -- Cambiamos al siguiente mes y año si es necesario
      in addDays (daysLeft - 1) (Date 1 nextMonth nextYear) -- Reiniciamos el día al 1 y seguimos


nextMonthYear :: String -> Int -> (String, Int)
nextMonthYear "December" year = ("January", year + 1)
nextMonthYear month year =
  case lookup month months of
    Just num -> case lookup (num + 1) (map (\(m, n) -> (n, m)) months) of
                  Just next -> (next, year)
                  Nothing -> error "Invalid month"
    Nothing -> error "Invalid month"

-- Cálculo de días entre fechas
daysFromStartOfYear :: Date -> Int
daysFromStartOfYear (Date d m y) =
  let monthDays = sum [daysInMonth month y | (month, _) <- takeWhile ((/= m) . fst) months]
  in monthDays + d

daysUntilEndOfYear :: Date -> Int
daysUntilEndOfYear (Date d m y) =
  let monthDays = sum [daysInMonth month y | (month, _) <- dropWhile ((/= m) . fst) months]
  in monthDays - d

daysInYear :: Int -> Int
daysInYear year = if isLeapYear year then 366 else 365

daysBetween :: Date -> Date -> Int
daysBetween (Date d1 m1 y1) (Date d2 m2 y2)
  | y1 == y2 = daysFromStartOfYear (Date d2 m2 y2) - daysFromStartOfYear (Date d1 m1 y1)
  | otherwise =
      let daysFirstYear = daysUntilEndOfYear (Date d1 m1 y1)
          daysLastYear = daysFromStartOfYear (Date d2 m2 y2)
          daysInMiddleYears = sum [daysInYear year | year <- [y1 + 1 .. y2 - 1]]
      in daysFirstYear + daysInMiddleYears + daysLastYear

-- Día de la semana
daysOfWeek :: [String]
daysOfWeek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

dayOfWeek :: Date -> String
dayOfWeek (Date d m y) =
  let refDate = Date 1 "January" 2025 -- Miércoles
      daysDiff = daysBetween refDate (Date d m y)
  in daysOfWeek !! ((2 + daysDiff) `mod` 7)

-- Validar si es día laborable
isWorkingDay :: Date -> [Date] -> Bool
isWorkingDay date holidays =
  let weekday = dayOfWeek date
  in weekday `notElem` ["Saturday", "Sunday"] && date `notElem` holidays

-- Próxima fecha laborable
nextWorkingDate :: Int -> Date -> [Date] -> Date
nextWorkingDate days startDate holidays =
  let targetDate = addDays days startDate
  in if isWorkingDay targetDate holidays
       then targetDate
       else nextWorkingDate 1 targetDate holidays

In [59]:
let startDate = Date 1 "January" 2025
    holidays = [Date 1 "January" 2025,Date 6 "January" 2025] -- Ejemplo: Día de Reyes
nextWorkingDate 1 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2025)

2 January 2025

13 January 2025

"Sunday"

In [60]:
let startDate = Date 30 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
--addDays 2 startDate
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2026)
"------------"
nextMonthYear "December" 2025
--2 - (daysInMonth "December" 2025 - 31)

2 January 2026

9 January 2026

"Monday"

"------------"

("January",2026)

In [61]:
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2027)
dayOfWeek (Date 27 "February" 2028)
dayOfWeek (Date 28 "February" 2028)
dayOfWeek (Date 29 "February" 2028)
dayOfWeek (Date 1 "March" 2028)

4 January 2027

11 January 2027

"Tuesday"

"Sunday"

"Monday"

"Tuesday"

"Wednesday"

In [62]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays

2 January 2026

12 January 2026

In [63]:
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays

4 January 2027

11 January 2027

In [64]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2026)
dayOfWeek (Date 12 "January" 2026)

2 January 2026

12 January 2026

"Monday"

"Monday"

In [65]:
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2027)
dayOfWeek (Date 27 "February" 2028)
dayOfWeek (Date 28 "February" 2028)
dayOfWeek (Date 29 "February" 2028)
dayOfWeek (Date 1 "March" 2028)

4 January 2027

11 January 2027

"Tuesday"

"Sunday"

"Monday"

"Tuesday"

"Wednesday"

In [66]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026, Date 6 "January" 2026] -- Ejemplo: Día de Reyes

-- Prueba de addDays
addDays 2 startDate


2 January 2026

### 10. A borrow User Publication Date Catalog function that borrows the publication starting in that date. If no copy is available on that date, the publication will be borrowed at the earliest possible date. It will return the updated Catalog. 

In [67]:
instance Eq Date where
  (Date d1 m1 y1) == (Date d2 m2 y2) =
    y1 == y2 && monthIndex m1 == monthIndex m2 && d1 == d2

instance Ord Date where
  compare (Date d1 m1 y1) (Date d2 m2 y2) =
    compare (y1, monthIndex m1, d1) (y2, monthIndex m2, d2)
    
holidays = [Date 15 "January" 2026, Date 16 "January" 2026]
    -- Convert month name to an index
monthIndex :: String -> Int
monthIndex month = case month of
    "January" -> 1; "February" -> 2; "March" -> 3; "April" -> 4; "May" -> 5
    "June" -> 6; "July" -> 7; "August" -> 8; "September" -> 9; "October" -> 10
    "November" -> 11; "December" -> 12; _ -> error "Invalid month"

In [68]:
findIndex :: (a -> Bool) -> [a] -> Maybe Int
findIndex p xs = findIndex' p xs 0
  where
    findIndex' _ [] _ = Nothing
    findIndex' p (x:xs) i
      | p x = Just i
      | otherwise = findIndex' p xs (i + 1)
      
maybeToList :: Maybe a -> [a]
maybeToList Nothing = []
maybeToList (Just x) = [x]

-- Duración del préstamo según el tipo de publicación y usuario
loanDuration :: User -> Publication -> Int
loanDuration (Student _) (Book _ _ _ _ _) = 30      -- 1 mes
loanDuration (Student _) (Journal _ _ _ _ _ _) = 15   -- 15 días
loanDuration (Student _) (DVD _ _ _ _ _ _ _) = 7     -- 1 semana
loanDuration (Professor _) (Book _ _ _ _ _) = 60   -- 2 meses
loanDuration (Professor _) (Journal _ _ _ _ _ _) = 30 -- 1 mes
loanDuration (Professor _) (DVD _ _ _ _ _ _ _) = 15  -- 15 días

findAvailableCopy :: Publication -> Date -> Maybe Int
findAvailableCopy publication currentDate =
  let copies = getCopies publication
  in findIndex (\copy -> not (isBorrowed copy) || isOverdue copy currentDate) copies


-- Determina si una copia está vencida y debe considerarse disponible
isOverdue :: Copy -> Date -> Bool
isOverdue copy currentDate =
  isBorrowed copy && case returnDate copy of
    Just rd -> rd < currentDate -- La fecha de devolución ya pasó
    Nothing -> False -- Si no hay fecha de devolución, no es vencida



-- Obtener la lista de copias de una publicación
getCopies :: Publication -> [Copy]
getCopies (Book _ _ _ _ copies) = copies
getCopies (Journal _ _ _ _ _ copies) = copies
getCopies (DVD _ _ _ _ _ _ copies) = copies

-- Actualizar una copia con la información del préstamo
updateCopy :: Copy -> User -> Date -> Date -> Copy
updateCopy copy user loanDate returnDate =
  copy { isBorrowed = True, borrower = Just user, loanDate = Just loanDate, returnDate = Just returnDate }

-- Actualizar la lista de copias de una publicación
updateCopies :: [Copy] -> Int -> Copy -> [Copy]
updateCopies copies index newCopy = take index copies ++ [newCopy] ++ drop (index + 1) copies

-- Actualizar una publicación con nuevas copias
updatePublication :: Publication -> [Copy] -> Publication
updatePublication (Book id author title pages _) copies = Book id author title pages copies
updatePublication (Journal id authors title releaseDate pages _) copies = Journal id authors title releaseDate pages copies
updatePublication (DVD id director actors title releaseDate duration _) copies = DVD id director actors title releaseDate duration copies

-- Reemplazar una publicación en el catálogo
replacePublication :: [Publication] -> Publication -> Publication -> [Publication]
replacePublication [] _ _ = []
replacePublication (x:xs) old new
  | x == old = new : xs
  | otherwise = x : replacePublication xs old new

-- Próxima fecha disponible
--nextAvailableDate :: Publication -> Date -> [Publication] -> Date
--nextAvailableDate publication date catalog =
  --let unavailableDates = concatMap (\copy -> maybeToList (returnDate copy)) (getCopies publication)
  --in head [nextDate | nextDate <- iterate (addDays 1) date, nextDate `notElem` unavailableDates]
nextAvailableDate :: Publication -> Date -> Date
nextAvailableDate publication date =
  let copies = getCopies publication
      returnDates = concatMap (\copy -> maybeToList (returnDate copy)) copies
  in case returnDates of
       [] -> error "No return dates available for this publication" -- Manejo de error explícito
       _  -> let earliestReturn = minimum returnDates -- La primera fecha en la que se devuelve una copia
                 adjustedDate = if date > earliestReturn then date else earliestReturn
             in adjustedDate

borrow :: User -> Publication -> Date -> [Publication] -> [Publication]
borrow user publication startDate catalog =
  let updatedCatalog = returnOverdueCopies user startDate catalog -- Paso 1: Devolver copias vencidas
  in if length (borrowedPublications user updatedCatalog) >= maxBorrowed user
        then updatedCatalog -- Paso 2: Si sigue excediendo el límite, no realizar el préstamo
        else -- Paso 3: Proceder con el préstamo
          let availableCopyIndex = findAvailableCopy publication (nextWorkingDate 0 startDate holidays)
          in case availableCopyIndex of
               Just index -> -- Hay una copia disponible
                 let loanPeriod = loanDuration user publication
                     returnDate = nextWorkingDate 0 (addDays loanPeriod (nextWorkingDate 0 startDate holidays)) holidays
                     updatedCopy = updateCopy (getCopies publication !! index) user (nextWorkingDate 0 startDate holidays) returnDate
                     updatedCopies = updateCopies (getCopies publication) index updatedCopy
                     updatedPublication = updatePublication publication updatedCopies
                 in replacePublication updatedCatalog publication updatedPublication
               Nothing -> -- No hay copias disponibles, buscar la próxima fecha disponible
                 borrow user publication (nextWorkingDate 1 (nextWorkingDate 0 startDate holidays) holidays) updatedCatalog

-- Devolver automáticamente las copias vencidas
returnOverdueCopies :: User -> Date -> [Publication] -> [Publication]
returnOverdueCopies user currentDate catalog = map (returnOverdueCopiesFromPublication user currentDate) catalog

-- Verificar y devolver copias vencidas dentro de una publicación
returnOverdueCopiesFromPublication :: User -> Date -> Publication -> Publication
returnOverdueCopiesFromPublication user currentDate publication =
  let updatedCopies = map (returnIfOverdue user currentDate) (getCopies publication)
  in updatePublication publication updatedCopies

-- Si una copia vencida pertenece al usuario, devolverla
returnIfOverdue :: User -> Date -> Copy -> Copy
returnIfOverdue user currentDate copy
  | isBorrowed copy && borrower copy == Just user && isOverdue copy currentDate =
      copy { isBorrowed = False, borrower = Nothing, loanDate = Nothing, returnDate = Nothing }
  | otherwise = copy


-- Determine the maximum number of publications a user can borrow
maxBorrowed :: User -> Int
maxBorrowed (Student _) = 3
maxBorrowed (Professor _) = 5




Line 15: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 16: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 17: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 18: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 19: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 20: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 73: Avoid lambda
Found:
\ copy -> maybeToList (returnDate copy)
Why not:
maybeToList . returnDateLine 77: Use max
Found:
if date > earliestReturn then date else earliestReturn
Why not:
max date earliestReturnLine 100: Eta reduce
Found:
returnOverdueCopies user currentDate catalog
  = map (returnOverdueCopiesFromPublication user currentDate) catalog
Why not:
returnOverdueCopies user currentDate
  = map (returnOverdueCopiesFromPublication user currentDate)

In [69]:
-- Usuarios
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))

-- Publicación con copia vencida
let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [ Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 30 "January" 2026)) ]

-- Fecha actual: después del vencimiento
let currentDate = Date 2 "January" 2026
findAvailableCopy book2 currentDate
findAvailableCopy book2 (Date 31 "January" 2026)
--isOverdue (Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 28 "December" 2025))) currentDate
--Date 2 "January" 2026 > (Date 28 "December" 2025)
-- Intento de préstamo
let result = borrow user1 book2 currentDate [book2]


Nothing

Just 0

In [70]:
result

[Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 2 February 2026 | Return Date: 4 March 2026
]

In [71]:
--Pruebas
-- Usuarios
holidays = [Date 15 "January" 2026, Date 16 "January" 2026]
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))

-- Publicaciones y copias
let book1 = Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200
                  [ Copy True (Just user1) (Just (Date 1 "December" 2025)) (Just (Date 31 "December" 2025))
                  , Copy False Nothing Nothing Nothing ]

let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [ Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 15 "December" 2025)) ]

-- Catálogo
let catalog = [book1, book2]


In [72]:
catalog
"---------------------"
borrow user2 book1 (Date 2 "December" 2025) catalog
"---------------------"
borrow user1 book2 (Date 2 "December" 2025) catalog

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Available
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 28 November 2025 | Return Date: 15 December 2025
]

"---------------------"

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 0/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 2 December 2025 | Return Date: 2 February 2026
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 28 November 2025 | Return Date: 15 December 2025
]

"---------------------"

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Available
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 16 December 2025 | Return Date: 19 January 2026
]

In [73]:
--prueba top
-- Users
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))
let user3 = Student (Person "Charlie" "Brown" (Date 1 "January" 2000))
-- Publications
let book1 = Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200
                  [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]

let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]
let book3 = Book "B3" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [Copy False Nothing Nothing Nothing,  Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]
-- Catalog
let catalog = [book1, book2, book3]
borrow user3 book2 (Date 2 "December" 2024) catalog
-- Borrow Attempt
{-let result1 = borrow user1 book1 (Date 2 "December" 2024) catalog
let result2 = borrow user2 book1 (Date 2 "December" 2024) result1
let user3 = Student (Person "Charlie" "Brown" (Date 1 "January" 2000))
let catalogWithLoans = [ Book "B3" (Person "Author" "Three" (Date 3 "March" 1990)) "Learn Haskell" 250
                           [ Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                             Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                             Copy False Nothing Nothing Nothing ]]
let result3 = borrow user3 book2 (Date 2 "December" 2024) catalogWithLoans
let user4 = Student (Person "Dana" "Scott" (Date 1 "January" 2000))
let catalogWithMaxLoans = [ Book "B4" (Person "Author" "Four" (Date 4 "April" 2000)) "Functional Magic" 150
                             [ Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                               Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                               Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)) ]]
let result4 = borrow user4 book2 (Date 2 "December" 2024) catalogWithMaxLoans


[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B3]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
]

In [74]:
result1
"------------------------------"
result2
"------------------------------"
result3
"------------------------------"
result4

Right [Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 2/3 available
Borrowed by Professor: Dr. Johnson (12 August 1975)
 | Loan Date: 10 November 2024 | Return Date: 10 December 2024
Available
Available
,Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
Available
Available
,Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
Available
Available
Available
]

"------------------------------"

: 

### 11. Functions to use I/O to add, remove, search and borrow publications, as well as function to show the catalog or the information about a user.

In [79]:
{-ffff
fffff-}
import System.IO

-- Main function to process commands from a file
processCommands :: FilePath -> FilePath -> Catalog -> IO ()
processCommands inputFile outputFile catalog = do
  handle <- openFile inputFile ReadMode -- Open the input file
  contents <- hGetContents handle       -- Read the contents
  let commands = lines contents         -- Split into lines
  finalCatalog <- executeCommands commands catalog -- Execute commands
  writeFile outputFile (prettyCatalog finalCatalog) -- Write the final catalog to the output file
  hClose handle                         -- Close the input file

-- Execute commands one by one
executeCommands :: [String] -> Catalog -> IO Catalog
executeCommands [] catalog = return catalog -- No more commands, return the final catalog
executeCommands (command:rest) catalog = do
  let (updatedCatalog, output) = executeCommand command catalog
  putStrLn output -- Print the result of the command to the console
  executeCommands rest updatedCatalog -- Continue with the rest of the commands

-- Execute a single command
executeCommand :: String -> Catalog -> (Catalog, String)
executeCommand command catalog =
  let parts = words command
      operation = read (head parts) :: Int -- First number indicates the operation
      args = tail parts                    -- Rest are arguments
  in case operation of
       1 -> -- Add to catalog
         let pub = parsePublication args
             updatedCatalog = addToCatalog pub catalog
         in (updatedCatalog, "Added to catalog:\n" ++ show pub)
       2 -> -- Remove from catalog
         let pub = parsePublication args
             result = removeFromCatalog pub catalog
         in case result of
              Right updatedCatalog -> (updatedCatalog, "Removed from catalog:\n" ++ show pub)
              Left errMsg          -> (catalog, errMsg)
       3 -> -- Search by author/title/date
         let (author, title, date) = parseSearchArgs args
             results = searchAuthorTitleDate author title date catalog
         in (catalog, "Search Results:\n" ++ unlines (map show results))
       4 -> -- Borrow a publication
         let user = parseUser (take 3 args)
             pub = parsePublication (drop 3 args)
             date = parseDate (drop 4 args)
             updatedCatalog = borrow user pub date catalog
         in (updatedCatalog, "Borrowed by " ++ show user ++ ":\n" ++ show pub)
       5 -> -- Show catalog
         (catalog, prettyCatalog catalog)
       6 -> -- Show user information
         let user = parseUser args
             borrowed = borrowedPublications user catalog
         in (catalog, "User Information:\n" ++ show user ++ "\nBorrowed:\n" ++ unlines (map show borrowed))
       _ -> (catalog, "Invalid command")
{-
-- Parse a publication from arguments
parsePublication :: [String] -> Publication
parsePublication args = case args of
  ("Book":id:authorFirst:authorLast:title:pages:[copies]) ->
    Book id (Person authorFirst authorLast (Date 1 "January" 1970)) title (read pages) (replicate (read copies) (Copy False Nothing Nothing Nothing))
  ("Journal":id:authors:title:date:pages:copies) ->
    Journal id (map parsePerson (words authors)) title (parseDate [date]) (read pages) (replicate (read copies) (Copy False Nothing Nothing Nothing))
  ("DVD":id:director:actors:title:date:duration:copies) ->
    DVD id (parsePerson [director]) (map parsePerson (words actors)) title (parseDate [date]) (read duration) (replicate (read copies) (Copy False Nothing Nothing Nothing))
  _ -> error "Invalid publication format"
-}
parsePublication :: [String] -> Publication
parsePublication args = case args of
  ("Book":id:authorFirst:authorLast:title:pages:copies) ->
    Book id (Person authorFirst authorLast (Date 1 "January" 1970)) title (read pages) (replicate (read copies :: Int) (Copy False Nothing Nothing Nothing))
  ("Journal":id:authors:title:date:pages:copies) ->
    Journal id (parsePeople authors) title (parseDate (words date)) (read pages) (replicate (read copies :: Int) (Copy False Nothing Nothing Nothing))
  ("DVD":id:director:actors:title:date:duration:copies) ->
    DVD id (parsePerson (words director)) (parsePeople actors) title (parseDate (words date)) (read duration) (replicate (read copies :: Int) (Copy False Nothing Nothing Nothing))
  _ -> error "Invalid publication format"

-- Parse a list of people (e.g., authors or actors)
parsePeople :: String -> [Person]
parsePeople str = map parsePerson (split ',' str)

-- Split a string by a delimiter (e.g., ',' for lists)
split :: Char -> String -> [String]
split _ [] = []
split delim str =
  let (before, remainder) = break (== delim) str
  in before : case remainder of
       [] -> []
       _  -> split delim (tail remainder)

-- Parse user arguments
parseUser :: [String] -> User
parseUser args = case args of
  ("Student":first:last:_) -> Student (Person first last (Date 1 "January" 2000))
  ("Professor":first:last:_) -> Professor (Person first last (Date 1 "January" 1980))
  _ -> error "Invalid user format"

-- Parse search arguments
parseSearchArgs :: [String] -> (Maybe Person, Maybe String, Maybe Date)
parseSearchArgs args = case args of
  ("Author":first:last:_) -> (Just (Person first last (Date 1 "January" 1970)), Nothing, Nothing)
  ("Title":title:_)       -> (Nothing, Just title, Nothing)
  ("Date":date:_)         -> (Nothing, Nothing, Just (parseDate [date]))
  _                       -> (Nothing, Nothing, Nothing)

-- Parse a date
parseDate :: [String] -> Date
parseDate [day, month, year] = Date (read day) month (read year)
parseDate _ = error "Invalid date format"

-- Parse a person
parsePerson :: [String] -> Person
parsePerson [first, last] = Person first last (Date 1 "January" 1970)
parsePerson _ = error "Invalid person format"


: 

In [89]:
handle <- openFile "ejemplo.txt" ReadMode -- Open the input file
contents <- hGetContents handle  
cont <- putStr contents          -- Read the contents
cont
--commands = lines contents         -- Split into lines

1 Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]) []
2 Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]) [Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))])]
3
4
5
6

()

In [84]:
contents
commands

"1 Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))]) []\n2 Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))]) [Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))])]\n3\n4\n5\n6"

["1 Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))]) []","2 Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))]) [Book \"B1\" (Person \"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))])]","3","4","5","6"]

In [85]:
import System.IO
import Text.Read (readMaybe)

-- Process the Add Publication command
processAddPublication :: FilePath -> IO ()
processAddPublication filePath = do
  handle <- openFile filePath ReadMode
  contents <- hGetContents handle
  let commands = lines contents -- Each line represents a command
  mapM_ processLine commands
  hClose handle

-- Process a single line
processLine :: String -> IO ()
processLine line = case parseAddPublication line of
  Just (pub, catalog) -> do
    let updatedCatalog = addToCatalog pub catalog
    putStrLn "Publication added successfully!"
    putStrLn $ "Updated Catalog:\n" ++ prettyCatalog updatedCatalog
  Nothing -> putStrLn "Error: Invalid input format for adding publication."

-- Parse the add publication command
parseAddPublication :: String -> Maybe (Publication, Catalog)
parseAddPublication input = readMaybe input :: Maybe (Publication, Catalog)


: 

In [90]:
instance Read Date where
  readsPrec _ input = case words input of
    [day, month, year] -> [(Date (read day) month (read year), "")]
    _ -> []

instance Read Person where
  readsPrec _ input = case words input of
    [firstName, lastName] -> [(Person firstName lastName (Date 1 "January" 1970), "")]
    _ -> []

instance Read Copy where
  readsPrec _ input =
    let (borrowedStr, rest1) = break (== ' ') input
        (borrowerStr, rest2) = break (== ' ') (tail rest1)
        (loanDateStr, returnDateStr) = break (== ' ') (tail rest2)
    in [(Copy (read borrowedStr)
              (readMaybe borrowerStr :: Maybe User)
              (readMaybe loanDateStr :: Maybe Date)
              (readMaybe returnDateStr :: Maybe Date), "")]

instance Read Publication where
  readsPrec _ input
    | "Book" `isPrefixOf` input =
        let (_, rest) = break (== ' ') input
            (id, rest1) = break (== ' ') rest
            (author, rest2) = break (== ' ') (tail rest1)
            (title, rest3) = break (== ' ') (tail rest2)
            (pages, rest4) = break (== ' ') (tail rest3)
            (copies, _) = break (== ' ') (tail rest4)
        in [(Book id (read author) title (read pages) (read copies), "")]
    | "Journal" `isPrefixOf` input =
        let (_, rest) = break (== ' ') input
            (id, rest1) = break (== ' ') rest
            (authors, rest2) = break (== ' ') (tail rest1)
            (title, rest3) = break (== ' ') (tail rest2)
            (releaseDate, rest4) = break (== ' ') (tail rest3)
            (pages, rest5) = break (== ' ') (tail rest4)
            (copies, _) = break (== ' ') (tail rest5)
        in [(Journal id (read authors) title (read releaseDate) (read pages) (read copies), "")]
    | "DVD" `isPrefixOf` input =
        let (_, rest) = break (== ' ') input
            (id, rest1) = break (== ' ') rest
            (director, rest2) = break (== ' ') (tail rest1)
            (actors, rest3) = break (== ' ') (tail rest2)
            (title, rest4) = break (== ' ') (tail rest3)
            (releaseDate, rest5) = break (== ' ') (tail rest4)
            (duration, rest6) = break (== ' ') (tail rest5)
            (copies, _) = break (== ' ') (tail rest6)
        in [(DVD id (read director) (read actors) title (read releaseDate) (read duration) (read copies), "")]
    | otherwise = []


: 

### 11. Nuevo intento


In [104]:
import System.IO
instance Read Date where
  readsPrec _ str =
    case words str of
      ["Date", d, m, y] ->
        let day = read d :: Int
            month = read m :: String
            year = read y :: Int
        in [(Date day month year, "")]
      _ -> []

instance Read Person where
  readsPrec _ str =
    let parsed = reads str :: [(String, String)] -- Parseamos el primer nombre
    in case parsed of
         [(first, rest1)] ->
           let [(last, rest2)] = reads rest1 :: [(String, String)] -- Parseamos el apellido
               [(date, rest3)] = reads rest2 :: [(Date, String)]  -- Parseamos la fecha de nacimiento
           in [(Person first last date, rest3)]
         _ -> []
         
instance Read Copy where
  readsPrec _ str =
    let parsed = reads str :: [(Bool, String)] -- Parseamos el estado de préstamo
    in case parsed of
         [(isBorrowed, rest1)] ->
           let [(maybeUser, rest2)] = reads rest1 :: [(Maybe User, String)] -- Usuario opcional
               [(maybeLoanDate, rest3)] = reads rest2 :: [(Maybe Date, String)] -- Fecha de préstamo opcional
               [(maybeReturnDate, rest4)] = reads rest3 :: [(Maybe Date, String)] -- Fecha de devolución opcional
           in [(Copy isBorrowed maybeUser maybeLoanDate maybeReturnDate, rest4)]
         _ -> []

instance Read User where
  readsPrec _ str =
    case words str of
      ("Student":rest) ->
        let [(person, rest1)] = reads (unwords rest) :: [(Person, String)]
        in [(Student person, rest1)]
      ("Professor":rest) ->
        let [(person, rest1)] = reads (unwords rest) :: [(Person, String)]
        in [(Professor person, rest1)]
      _ -> []

instance Read Publication where
  readsPrec _ str =
    case words str of
      ("Book":id:rest) ->
        let [(author, rest1)] = reads (unwords rest) :: [(Person, String)]
            [(title, rest2)] = reads rest1 :: [(String, String)]
            [(pages, rest3)] = reads rest2 :: [(Int, String)]
            [(copies, rest4)] = reads rest3 :: [([Copy], String)]
        in [(Book id author title pages copies, rest4)]
      ("Journal":id:rest) ->
        let [(authors, rest1)] = reads (unwords rest) :: [([Person], String)]
            [(title, rest2)] = reads rest1 :: [(String, String)]
            [(releaseDate, rest3)] = reads rest2 :: [(Date, String)]
            [(pages, rest4)] = reads rest3 :: [(Int, String)]
            [(copies, rest5)] = reads rest4 :: [([Copy], String)]
        in [(Journal id authors title releaseDate pages copies, rest5)]
      ("DVD":id:rest) ->
        let [(director, rest1)] = reads (unwords rest) :: [(Person, String)]
            [(actors, rest2)] = reads rest1 :: [([Person], String)]
            [(title, rest3)] = reads rest2 :: [(String, String)]
            [(releaseDate, rest4)] = reads rest3 :: [(Date, String)]
            [(duration, rest5)] = reads rest4 :: [(Int, String)]
            [(copies, rest6)] = reads rest5 :: [([Copy], String)]
        in [(DVD id director actors title releaseDate duration copies, rest6)]
      _ -> []

In [172]:
import System.IO

-- Función para agregar una publicación desde un archivo
addPublicationFromFile :: FilePath -> Catalog -> IO Catalog
addPublicationFromFile filePath catalog = do
  contents <- readFile filePath
  let linesOfFile = lines contents
  case linesOfFile of
    (header:args) -> do
      let publication = parsePublication header args
      return $ addToCatalog publication catalog
    _ -> do
      putStrLn "Error: Archivo vacío o mal formateado"
      return catalog

-- Función para parsear una publicación desde las líneas del archivo
parsePublication :: String -> [String] -> Publication
parsePublication header args =
  case words header of
    ["1", "Book", bookId, author, title, pages, copies] ->
      Book bookId (parsePerson author) (init title) (read pages) (parseCopies copies)
    ["1", "Journal", journalId, authors, title, releaseDate, pages, copies] ->
      Journal journalId (parsePersons authors) (init title) (parseDate releaseDate) (read pages) (parseCopies copies)
    ["1", "DVD", dvdId, director, actors, title, releaseDate, duration, copies] ->
      DVD dvdId (parsePerson director) (parsePersons actors) (init title) (parseDate releaseDate) (read duration) (parseCopies copies)
    _ -> error "Formato de publicación no reconocido"

-- Parseo de Person
parsePerson :: String -> Person
parsePerson str =
  let (first, last, date) = read str :: (String, String, Date)
  in Person first last date

-- Parseo de [Person]
parsePersons :: String -> [Person]
parsePersons str = read str :: [Person]

-- Parseo de Date
parseDate :: String -> Date
parseDate str = read str :: Date

-- Parseo de [Copy]
parseCopies :: String -> [Copy]
parseCopies str = read str :: [Copy]

-- Función para agregar al catálogo (ya definida por ti)

-- Prueba del sistema
main :: IO ()
main = do
  let catalog = [] -- Catálogo inicial vacío
  catalog' <- addPublicationFromFile "publications.txt" catalog
  print catalog'


: 

In [ ]:
  contents <- readFile "publications.txt"
  let linesOfFile = lines contents
  case linesOfFile of
    (header:args) -> do
      let publication = parsePublication header args
      putStrLn $ (words header) !! 6
      return $ catalog
    _ -> do
      putStrLn "Error: Archivo vacío o mal formateado"
      return catalog

(Date
[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B3]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
]

In [170]:
addPublicationFromFile "publications.txt" catalog

: 

In [108]:
--Prueba
contents <- readFile "publications.txt"
lines contents

["1 Book \"B1\" (\"Author\" \"One\" (Date 1 \"January\" 1970)) \"Functional Programming\" 200 [Copy False Nothing Nothing Nothing, Copy True (Just (Student (Person \"User\" \"Three\" (Date 1 \"January\" 1990)))) (Just (Date 1 \"December\" 2024)) (Just (Date 31 \"December\" 2024))]"]

In [110]:
import System.IO
-- Instancias `Read` necesarias
instance Read Date where
  readsPrec _ str =
    case words str of
      ["Date", d, m, y] -> [(Date (read d) (read m) (read y), "")]
      _ -> []

instance Read Person where
  readsPrec _ str =
    case reads str :: [(String, String)] of
      [(first, rest1)] ->
        case reads rest1 :: [(String, String)] of
          [(last, rest2)] ->
            case reads rest2 :: [(Date, String)] of
              [(date, rest3)] -> [(Person first last date, rest3)]
              _ -> []
          _ -> []
      _ -> []

instance Read Copy where
  readsPrec _ str =
    case reads str :: [(Bool, String)] of
      [(isBorrowed, rest1)] ->
        case reads rest1 :: [(Maybe User, String)] of
          [(maybeUser, rest2)] ->
            case reads rest2 :: [(Maybe Date, String)] of
              [(maybeLoanDate, rest3)] ->
                case reads rest3 :: [(Maybe Date, String)] of
                  [(maybeReturnDate, rest4)] ->
                    [(Copy isBorrowed maybeUser maybeLoanDate maybeReturnDate, rest4)]
                  _ -> []
              _ -> []
          _ -> []
      _ -> []

instance Read User where
  readsPrec _ str =
    case words str of
      ("Student":rest) -> [(Student (read (unwords rest)), "")]
      ("Professor":rest) -> [(Professor (read (unwords rest)), "")]
      _ -> []

instance Read Publication where
  readsPrec _ str =
    case words str of
      ("Book":id:rest) ->
        let (author, rest1) = parsePerson rest
            (title, rest2) = parseString rest1
            (pages, rest3) = parseInt rest2
            (copies, _) = parseCopies rest3
        in [(Book id author title pages copies, "")]
      ("Journal":id:rest) ->
        let (authors, rest1) = parsePersons rest
            (title, rest2) = parseString rest1
            (releaseDate, rest3) = parseDate rest2
            (pages, rest4) = parseInt rest3
            (copies, _) = parseCopies rest4
        in [(Journal id authors title releaseDate pages copies, "")]
      ("DVD":id:rest) ->
        let (director, rest1) = parsePerson rest
            (actors, rest2) = parsePersons rest1
            (title, rest3) = parseString rest2
            (releaseDate, rest4) = parseDate rest3
            (duration, rest5) = parseInt rest4
            (copies, _) = parseCopies rest5
        in [(DVD id director actors title releaseDate duration copies, "")]
      _ -> []

-- Función para leer la publicación y el catálogo desde un archivo
addPublicationFromFile :: FilePath -> IO Catalog
addPublicationFromFile filePath = do
  contents <- readFile filePath
  let line = head (lines contents) -- Leemos solo la primera línea
  case words line of
    ("1":rest) ->
      let (publication, restCatalog) = parsePublicationAndCatalog rest
      in return $ addToCatalog publication (read restCatalog :: Catalog)
    _ -> do
      putStrLn "Error: Formato de archivo no válido"
      return []

-- Función para parsear la publicación y el catálogo
parsePublicationAndCatalog :: [String] -> (Publication, String)
parsePublicationAndCatalog wordsList =
  let (pubStr, catalogStr) = break (== "]") wordsList
      pubStrJoined = unwords (pubStr ++ ["]"])
      catalogStrJoined = unwords (drop 1 catalogStr) -- Saltamos el "]"
  in (read pubStrJoined :: Publication, catalogStrJoined)

-- Función `addToCatalog` proporcionada por el usuario
addToCatalog :: Publication -> Catalog -> Catalog
addToCatalog pub [] = [pub]
addToCatalog pub (p:ps)
  | samePublication pub p = mergeCopies pub p : ps
  | otherwise             = p : addToCatalog pub ps
  where
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    mergeCopies (Book id1 author1 title1 pages1 copies1) (Book _ _ _ _ copies2) =
      Book id1 author1 title1 pages1 (copies1 ++ copies2)
    mergeCopies (Journal id1 authors1 title1 release1 pages1 copies1) (Journal _ _ _ _ _ copies2) =
      Journal id1 authors1 title1 release1 pages1 (copies1 ++ copies2)
    mergeCopies (DVD id1 director1 actors1 title1 release1 duration1 copies1) (DVD _ _ _ _ _ _ copies2) =
      DVD id1 director1 actors1 title1 release1 duration1 (copies1 ++ copies2)
    mergeCopies pub1 _ = pub1


-- Prueba del sistema
addPublicationFromFile "publications.txt"
 


: 

In [165]:
import System.IO

-- Función para leer publicación y catálogo desde un archivo
addPublicationFromFile :: FilePath -> IO Catalog
addPublicationFromFile filePath = do
  contents <- readFile filePath
  let line = head (lines contents) -- Leemos solo la primera línea del archivo
  case words line of
    ("1":rest) ->
      let (publication, catalogStr) = splitPublicationAndCatalog (unwords rest)
          catalog = read catalogStr :: Catalog
      in return $ addToCatalog (read publication :: Publication) catalog
    _ -> do
      putStrLn "Error: Formato de archivo no válido"
      return []

-- Función para dividir publicación y catálogo en la línea
splitPublicationAndCatalog :: String -> (String, String)
splitPublicationAndCatalog line =
  let (pubStr, star:catalogStart) = break (== '*') line -- Dividimos en la publicación y el catálogo
  in (pubStr, catalogStart)

-- Función `addToCatalog` proporcionada por el usuario
addToCatalog :: Publication -> Catalog -> Catalog
addToCatalog pub [] = [pub]
addToCatalog pub (p:ps)
  | samePublication pub p = mergeCopies pub p : ps
  | otherwise             = p : addToCatalog pub ps
  where
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    mergeCopies (Book id1 author1 title1 pages1 copies1) (Book _ _ _ _ copies2) =
      Book id1 author1 title1 pages1 (copies1 ++ copies2)
    mergeCopies (Journal id1 authors1 title1 release1 pages1 copies1) (Journal _ _ _ _ _ copies2) =
      Journal id1 authors1 title1 release1 pages1 (copies1 ++ copies2)
    mergeCopies (DVD id1 director1 actors1 title1 release1 duration1 copies1) (DVD _ _ _ _ _ _ copies2) =
      DVD id1 director1 actors1 title1 release1 duration1 (copies1 ++ copies2)
    mergeCopies pub1 _ = pub1

-- Instancias `Read` necesarias
instance Read Date where
  readsPrec _ str =
    case words str of
      ["Date", d, m, y] -> [(Date (read d) (read m) (read y), "")]
      _ -> []

instance Read Person where
  readsPrec _ str =
    case reads str :: [(String, String)] of
      [(first, rest1)] ->
        case reads rest1 :: [(String, String)] of
          [(last, rest2)] ->
            case reads rest2 :: [(Date, String)] of
              [(date, rest3)] -> [(Person first last date, rest3)]
              _ -> []
          _ -> []
      _ -> []

instance Read Copy where
  readsPrec _ str =
    case reads str :: [(Bool, String)] of
      [(isBorrowed, rest1)] ->
        case reads rest1 :: [(Maybe User, String)] of
          [(maybeUser, rest2)] ->
            case reads rest2 :: [(Maybe Date, String)] of
              [(maybeLoanDate, rest3)] ->
                case reads rest3 :: [(Maybe Date, String)] of
                  [(maybeReturnDate, rest4)] ->
                    [(Copy isBorrowed maybeUser maybeLoanDate maybeReturnDate, rest4)]
                  _ -> []
              _ -> []
          _ -> []
      _ -> []

instance Read User where
  readsPrec _ str =
    case words str of
      ("Student":rest) -> [(Student (read (unwords rest)), "")]
      ("Professor":rest) -> [(Professor (read (unwords rest)), "")]
      _ -> []

instance Read Publication where
  readsPrec _ str =
    case words str of
      ("Book":id:rest) ->
        let (author, rest1) = parseAuthorAndRest rest
            (title, rest2) = parseStringAndRest rest1
            (pages, rest3) = parseIntAndRest rest2
            (copies, _) = parseCopiesAndRest rest3
        in [(Book id author title pages copies, "")]
      ("Journal":id:rest) ->
        let (authors, rest1) = parseAuthorsAndRest rest
            (title, rest2) = parseStringAndRest rest1
            (releaseDate, rest3) = parseDateAndRest rest2
            (pages, rest4) = parseIntAndRest rest3
            (copies, _) = parseCopiesAndRest rest4
        in [(Journal id authors title releaseDate pages copies, "")]
      ("DVD":id:rest) ->
        let (director, rest1) = parseAuthorAndRest rest
            (actors, rest2) = parseAuthorsAndRest rest1
            (title, rest3) = parseStringAndRest rest2
            (releaseDate, rest4) = parseDateAndRest rest3
            (duration, rest5) = parseIntAndRest rest4
            (copies, _) = parseCopiesAndRest rest5
        in [(DVD id director actors title releaseDate duration copies, "")]
      _ -> []

-- Funciones auxiliares de parseo
-- Funciones auxiliares
parseAuthorAndRest :: [String] -> (Person, [String])
parseAuthorAndRest xs =
  let (personStr, rest) = break (== ")") xs
      person = read (unwords (personStr ++ [")"])) :: Person
  in (person, tail rest)

parseAuthorsAndRest :: [String] -> ([Person], [String])
parseAuthorsAndRest xs =
  let (authorsStr, rest) = break (== "]") xs
      authors = read (unwords (authorsStr ++ ["]"])) :: [Person]
  in (authors, tail rest)

parseStringAndRest :: [String] -> (String, [String])
parseStringAndRest (x:xs) = (read x :: String, xs)

parseDateAndRest :: [String] -> (Date, [String])
parseDateAndRest xs =
  let (dateStr, rest) = break (== ")") xs
      date = read (unwords (dateStr ++ [")"])) :: Date
  in (date, tail rest)

parseIntAndRest :: [String] -> (Int, [String])
parseIntAndRest (x:xs) = (read x :: Int, xs)

parseCopiesAndRest :: [String] -> ([Copy], [String])
parseCopiesAndRest xs =
  let (copiesStr, rest) = break (== "]") xs
      copies = read (unwords (copiesStr ++ ["]"])) :: [Copy]
  in (copies, tail rest)

-- Prueba del sistema
addPublicationFromFile "publications.txt"
--catalog


: 

In [166]:
contents <- readFile "publications.txt"
line = head (lines contents) -- Leemos solo la primera línea del archivo
words line
splitPublicationAndCatalog (unwords rest)
--catalog = read catalogStr :: Catalog

["1","Book","\"B1\"","(Person","\"Author\"","\"One\"","(Date","1","\"January\"","1970))","\"Functional","Programming\"","200","[Copy","False","Nothing","Nothing","Nothing]*[Book","\"B2\"","(Person","\"Author\"","\"Two\"","(Date","2","\"February\"","1980))","\"Advanced","Programming\"","300","[Copy","True","Nothing","Nothing","Nothing]]"]

: 

In [156]:
contents <- readFile "publications.txt"
let line = head (lines contents) -- Leemos solo la primera línea del archivo
case words line of
    ("1":rest) ->
        let (publication, catalogStr) = splitPublicationAndCatalog (unwords rest)
        --catalog = read catalogStr :: Catalog

: 

In [157]:
("1":rest) = line
rest
unwords rest
line
('1':rest)

: 

In [158]:
splitPublicationAndCatalog (unwords rest)

: 

In [159]:
parsePublicationAndCatalog rest

: 

In [164]:
contents <- readFile "publications.txt"
let line = head (lines contents) -- Leemos solo la primera línea del archivo
case words line of
    ("1":rest) ->
      let (publication, catalogStr) = splitPublicationAndCatalog (unwords rest)
          catalog = read catalogStr :: Catalog
      in return $ catalogStr
    _ -> do
      putStrLn "Error: Formato de archivo no válido"
      return []

"[Book \"B2\" (Person \"Author\" \"Two\" (Date 2 \"February\" 1980)) \"Advanced Programming\" 300 [Copy True Nothing Nothing Nothing]]"

In [161]:
publication
catalogStr

: 